In [1]:
import pyspark
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
pyspark.__file__

'/opt/homebrew/Cellar/apache-spark/3.5.0/libexec/python/pyspark/__init__.py'

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/24 15:39:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
print("Spark Version:", spark.version)

Spark Version: 3.5.0


In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [ ]:
!gunzip fhv_tripdata_2019-10.csv.gz

In [ ]:
!ls

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [ ]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [ ]:
df.show()

In [ ]:
df = df.repartition(6)

In [ ]:
df.write.parquet('fhv/2019/10/')

In [ ]:
df.printSchema()

In [7]:
df = spark.read.parquet('fhv/2019/10/')

In [ ]:
df.show()

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

In [9]:
df.createOrReplaceTempView('fhv_2019_10')

In [ ]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

In [ ]:
df \
    .withColumn('duration', (df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long'))/3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

In [ ]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropOff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

In [ ]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropOff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration,
    MAX(dropOff_datetime) AS dropoff_datetime,
    MAX(pickup_datetime) AS pickup_datetime
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 20;
""").show()

- 631,152.50 Hours Hence the error in data, the answer according to the computing result=631,152.50 Hours

In [ ]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [7]:
df.write.parquet('zones')

In [5]:
df_zones = spark.read.parquet('zones')

In [6]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [8]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [10]:
df_zones.createOrReplaceTempView('zones')

In [13]:
spark.sql("""
SELECT
    z.Zone
FROM
    fhv_2019_10 fhv
JOIN
    zones z
ON
    fhv.PULocationID = z.LocationID
GROUP BY
    z.Zone
ORDER BY
    COUNT(*) ASC

""").show()

+--------------------+
|                Zone|
+--------------------+
|         Jamaica Bay|
|Governor's Island...|
| Green-Wood Cemetery|
|       Broad Channel|
|     Highbridge Park|
|        Battery Park|
|Saint Michaels Ce...|
|Breezy Point/Fort...|
|Marine Park/Floyd...|
|        Astoria Park|
|    Inwood Hill Park|
|       Willets Point|
|Forest Park/Highl...|
|  Brooklyn Navy Yard|
|        Crotona Park|
|        Country Club|
|     Freshkills Park|
|       Prospect Park|
|     Columbia Street|
|  South Williamsburg|
+--------------------+
only showing top 20 rows

